In [ ]:
#!/usr/bin/python3
import numpy as np
import cv2 as cv
import torch
import matplotlib
import tkinter
import matplotlib.pyplot as plt
import pandas as pd
print(torch.__version__)
# print(plt.get_backend())
model = torch.hub.load('ultralytics/yolov5', 'yolov5s',
                       pretrained=True)  # this line changes matplotlib backend beware lol
# imgs = ['https://ultralytics.com/images/zidane.jpg']
imgs = []
cap = cv.VideoCapture('test_vid.mp4')
# print(cap)
# if cap.isOpened()==False:
# print("l lag gaye")
count = 0
# matplotlib.rcParams['backend'] = 'GTK3Agg'
df_lists = []
while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    if ret:
        # Display the resulting frameq
        # cv.imshow('Frame',frame)
        if count % 5 == 0:
            imgs.append(frame)
        # Press Q on keyboard to  exit
        if cv.waitKey(25) & 0xFF == ord('q'):
            break
    # Break the loop
    else:
        break
    count += 1
    if count % 50 == 0:
        results = model(imgs)
        # results.print()
        df_lists += results.pandas().xyxy  # list of lists here lol
        imgs = []
        # break
# print(result.bbox.x)
# print(imgs[0].shape)
# results = model(imgs)
df_lists += results.pandas().xyxy
print(len(df_lists))
agg_rslt = pd.concat(df_lists, axis=0)  # axis 0 for columns
temp = agg_rslt.loc[agg_rslt['class'] == 0]
print(agg_rslt.describe())
# agg_rslt = agg_rslt.drop(agg_rslt[agg_rslt['class']!=0].index)
print(temp.describe())

# matplotlib.use('GTK3Agg',force=True)
# plt.imshow(cv.cvtColor(imgs[25], cv.COLOR_BGR2RGB))
# plt.show()
# Results
# print(plt.get_backend())
# gui_env = [i for i in matplotlib.rcsetup.interactive_bk]
# non_gui_backends = matplotlib.rcsetup.non_interactive_bk
# print ("Non Gui backends are:", non_gui_backends)
# print ("Gui backends I will test for", gui_env)
# results.save()  # or .show()
# print(results.xyxy[0])  # img1 predictions (tensor)
# results.pandas().xyxy[0]  # img1 predictions (pandas)


Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2022-3-19 torch 1.10.0+cu111 CPU



1.10.0+cu111


Fusing layers... 
Model Summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


60
             xmin        ymin        xmax        ymax  confidence       class
count  104.000000  104.000000  104.000000  104.000000  104.000000  104.000000
mean   431.510308  199.242688  708.519581  476.047578    0.658791   24.673077
std    253.419825  129.951361  101.285510    4.339869    0.291966   28.967153
min    209.627884   75.259277  582.897766  455.954468    0.254338    0.000000
25%    215.846134   93.776867  616.895279  475.839699    0.346464    0.000000
50%    227.223770  116.917751  666.516235  477.267868    0.843887    0.000000
75%    710.447235  259.265816  830.754715  478.325943    0.934143   58.000000
max    824.077271  439.333069  854.000000  480.000000    0.950545   65.000000
             xmin        ymin        xmax        ymax  confidence  class
count   60.000000   60.000000   60.000000   60.000000   60.000000   60.0
mean   218.522840   98.813993  629.637890  477.120577    0.903090    0.0
std      6.440500   13.326191   28.548150    1.274669    0.047419    0.0
min

In [ ]:
from google.colab import data_table
data_table.enable_dataframe_formatter()
from __future__ import print_function

agg_rslt

,xmin,ymin,xmax,ymax,confidence,class,name
0,209.627884,127.019165,596.465759,476.308685,0.936530,0,person
0,211.452820,127.799026,589.760864,476.153442,0.935713,0,person
1,787.256897,415.195007,850.430603,478.773132,0.359676,58,potted plant
0,211.211411,116.412979,583.511353,476.256958,0.943678,0,person
1,792.231567,415.491791,847.584534,478.000397,0.350202,58,potted plant
...,...,...,...,...,...,...,...
0,224.561218,93.631996,659.464661,476.759033,0.874497,0,person
1,709.308044,254.579697,853.330139,477.288177,0.401816,58,potted plant
0,225.957001,88.099190,670.837097,474.898895,0.879064,0,person
0,222.398560,86.952446,693.470886,475.736450,0.888453,0,person


In [ ]:
import cv2
import math
import argparse
from google.colab.patches import cv2_imshow

def highlightFace(net, frame, conf_threshold=0.7):
    frameOpencvDnn=frame.copy()
    frameHeight=frameOpencvDnn.shape[0]
    frameWidth=frameOpencvDnn.shape[1]
    blob=cv2.dnn.blobFromImage(frameOpencvDnn, 1.0, (300, 300), [104, 117, 123], True, False)

    net.setInput(blob)
    detections=net.forward()
    faceBoxes=[]
    for i in range(detections.shape[2]):
        confidence=detections[0,0,i,2]
        if confidence>conf_threshold:
            x1=int(detections[0,0,i,3]*frameWidth)
            y1=int(detections[0,0,i,4]*frameHeight)
            x2=int(detections[0,0,i,5]*frameWidth)
            y2=int(detections[0,0,i,6]*frameHeight)
            faceBoxes.append([x1,y1,x2,y2])
            cv2.rectangle(frameOpencvDnn, (x1,y1), (x2,y2), (0,255,0), int(round(frameHeight/150)), 8)
    return frameOpencvDnn,faceBoxes


faceProto="opencv_face_detector.pbtxt"
faceModel="opencv_face_detector_uint8.pb"
ageProto="age_deploy.prototxt"
ageModel="age_net.caffemodel"
genderProto="gender_deploy.prototxt"
genderModel="gender_net.caffemodel"

MODEL_MEAN_VALUES=(78.4263377603, 87.7689143744, 114.895847746)
ageList=['(0-2)', '(4-6)', '(8-12)', '(15-20)', '(25-32)', '(38-43)', '(48-53)', '(60-100)']
genderList=['Male','Female']

faceNet=cv2.dnn.readNet(faceModel,faceProto)
ageNet=cv2.dnn.readNet(ageModel,ageProto)
genderNet=cv2.dnn.readNet(genderModel,genderProto)

# video=cv2.VideoCapture(args.image if args.image else 0)
video = cv2.VideoCapture('test_vid.mp4')
padding=20
count = 0
while video.isOpened() :
    hasFrame,frame=video.read()
    count = count + 1
    if not hasFrame:
        cv2.waitKey()
        break
    
    resultImg,faceBoxes=highlightFace(faceNet,frame)
    if not faceBoxes:
        print("No face detected")

    for faceBox in faceBoxes:
        face=frame[max(0,faceBox[1]-padding):
                   min(faceBox[3]+padding,frame.shape[0]-1),max(0,faceBox[0]-padding)
                   :min(faceBox[2]+padding, frame.shape[1]-1)]

        blob=cv2.dnn.blobFromImage(face, 1.0, (227,227), MODEL_MEAN_VALUES, swapRB=False)
        genderNet.setInput(blob)
        genderPreds=genderNet.forward()
        gender=genderList[genderPreds[0].argmax()]
        print(f'Gender: {gender}')

        ageNet.setInput(blob)
        agePreds=ageNet.forward()
        age=ageList[agePreds[0].argmax()]
        print(f'Age: {age[1:-1]} years')

        cv2.putText(resultImg, f'{gender}, {age}', (faceBox[0], faceBox[1]-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,255,255), 2, cv2.LINE_AA)
        cv2_imshow(resultImg)